In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_circles, make_regression, make_swiss_roll
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from IPython.display import display

%matplotlib inline

In [ ]:
def plot_2d_function(X1, X2, P, figsize, title):
    
    """
    
    Функция для визуализации решающей функции
    
    X1 – матрица значений по признаку x1
    X2 – матрица значений по признаку x2
    P – решающая функция
    
    title – заголовок картинки
    """
    
    plt.figure(figsize=figsize)
    plt.contourf(X1, X2, P, cmap='coolwarm', levels=2)
    plt.title(title)
    plt.xlabel('$X1$')
    plt.ylabel('$X2$');
    
    
def plot_clf_dataset(X, y, x1_lim, x2_lim, figsize):
    
    """
    Функция для визуализации датасета задачи классификации
    
    X - матрица объектов с как минимум двумя признаками
    y – ответы на объектах
    x1_lim – ограничения визуализации по x1
    x2_lim – ограничения визуализации по x2
    figsize – размер картинки
    """
    
    plt.figure(figsize=figsize)
    plt.scatter(X[:, 0], X[:, 1], c=y)
    plt.xlim(x1_lim)
    plt.ylim(x2_lim)
    plt.xlabel('$x1$')
    plt.ylabel('$x2$');
    
    
def plot_reg_dataset(X, y, figsize):
    
    """
    Функция для визуализации датасета задачи регрессии
    
    X - матрица объектов с как минимум одним признаком
    y – ответы на объектах
    figsize – размер картинки
    
    """
        
    plt.figure(figsize=figsize)
    plt.scatter(X[:, 0], y)
    plt.xlabel('$x$')
    plt.ylabel('$y$');
    
    
def get_decision_function(x1, x2, model, x1_lim, x2_lim, num=50):
    
    """
    
    Функция для получения предикта решающего дерева в 2d
    
    x1 – значение признаков по x1
    x2 – значение признаков по x2
    
    model – обученная модель
    x1_lim – ограничения визуализации по x1
    x2_lim – ограничения визуализации по x2
    num – разрешение решающей функции
    
    """

    x1_grid = np.linspace(*x1_lim, num=num)
    x2_grid = np.linspace(*x2_lim, num=num)

    X1, X2 = np.meshgrid(x1_grid, x2_grid)

    P = np.zeros_like(X1)

    for i in range(num):
        for j in range(num):
            P[i, j] = model.predict([[X1[i, j],  X2[i, j]]])
            
    return X1, X2, P

* [Дерево классификации](#Дерево-классификации)
* [Дерево регрессии](#Дерево-регрессии)

* [Строим решающее дерево для небольшой выборки классификации](#Строим-решающее-дерево-для-небольшой-выборки-классификации)
* [Строим решающее дерево для небольшой выборки регрессии](#Строим-решающее-дерево-для-небольшой-выборки-регрессии)
* [Регуляризуем дерево на датасете титаника](#Регуляризуем-дерево-на-датасете-титаника)

In [ ]:
FIG_SIZE = (12, 8)
X1_LIM = (-1, 1.2)
X2_LIM = (-1.1, 1.1)

## Дерево классификации

In [ ]:
X, y = make_circles(n_samples=100)
plot_clf_dataset(X, y, x1_lim=X1_LIM, x2_lim=X2_LIM, figsize=FIG_SIZE)

In [ ]:
clf_tree = DecisionTreeClassifier()
clf_tree.fit(X, y)

plt.figure(figsize=(7, 7))
plot_tree(clf_tree, feature_names=['x1', 'x2'], filled=True, );

In [ ]:
X1, X2, P = get_decision_function(
    X[:, 0], X[:, 1], model=clf_tree, x1_lim=X1_LIM, x2_lim=X2_LIM
)
plot_2d_function(X1, X2, P, figsize=FIG_SIZE, title='Решающая функция для decision tree')

plt.scatter(X[:, 0], X[:, 1], c=y)
plt.xlim(X1_LIM)
plt.ylim(X2_LIM);

## Дерево регрессии

In [ ]:
X, y = make_regression(n_samples=100, n_features=1, noise=10, random_state=1)

# для нелинейности
# x_min, x_max = -3, 3
# X = np.linspace(x_min, x_max, num=20).reshape(-1, 1)
# y = np.sin(X)

plt.figure(figsize=FIG_SIZE)
plt.scatter(X.flatten(), y)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Regression data');

In [ ]:
reg_tree = DecisionTreeRegressor(max_depth=5, criterion='absolute_error')
reg_tree.fit(X, y)

In [ ]:
x_min, x_max = X.min()-1, X.max()+1
X_val = np.linspace(x_min, x_max, num=1000).reshape(-1, 1)

predict = reg_tree.predict(X_val)

plt.figure(figsize=FIG_SIZE)
plt.scatter(X.flatten(), y)
plt.plot(X_val, predict, 'r', label='dt predict')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.title('Decision tree for regression');

### Экстраполяция

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
x_min, x_max = X.min() - 1, X.max() + 1
X_val = np.linspace(x_min, x_max, num=1000).reshape(-1, 1)

predict = reg_tree.predict(X_val)
linreg = LinearRegression().fit(X, y)
predict_ln = linreg.predict(X_val)

plt.figure(figsize=FIG_SIZE)
plt.scatter(X.flatten(), y)
plt.plot(X_val, predict, 'r', label='dt predict')
plt.plot(X_val, predict_ln, 'g', label='ln predict')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.title('Decision tree for regression');

## Строим решающее дерево для небольшой выборки классификации

$$Q(X_m, j, t) = \frac{N_l}{N_m}H_l + \frac{N_r}{N_m}H_r$$
 
$H_l$ – критерий информативности для левой подвыборки;
$H_r$ – критерий информативности для правой подвыборки; 

$N_m$ – общее количество объектов;
$N_l$ – количество объектов в левой подвыборке;
$N_r$ – количество объектов в левой подвыборке.

**Критерии информативности в задачах классификации**

Gini: 
$$H = \sum_k p_k(1-p_k)$$

Entropy:
$$H = -\sum_k p_k log(p_k)$$

где $p_k$ – доля класса $k$ в данной подвыборке.

**Задание**

* Построить решающее дерево глубины 2 для задачи классификации аналитически (посчитав критерий информативности для всех возможных сплитов на листочке или с помощью функции на python);
* Сравнить полученный результат с реализацией в sklearn с помощью `graphviz`.

In [ ]:
def Q():
    pass

def H():
    pass


In [ ]:
X = np.array([
    [0.5, 3],
    [3.5, 1],
    [3.9, 4],
    [2, 2],
    [1.5, 1.2]
])

y = [0, 1, 0, 1, 0]

plot_clf_dataset(X, y, x1_lim=(0, 4),  x2_lim=(0, 5), figsize=FIG_SIZE)

In [ ]:
data = pd.DataFrame(X, columns=['x1', 'x2'])
data['y'] = y

In [ ]:
sorted_x1 = data.sort_values(by='x1')

In [ ]:
for i in range(1, sorted_x1.shape[0]):
    left = sorted_x1[:i]
    right = sorted_x1[i:]
    display(Q(left, right))
    break

In [ ]:
def H(data_part):
    p_ks = data_part['y'].value_counts(normalize=True)
    p_0 = p_ks.loc[0] if (data_part['y'] == 0).sum() > 0 else 0
    p_1 = p_ks.loc[1] if (data_part['y'] == 1).sum() > 0 else 0
    
    return p_0 * (1 - p_0) + p_1 * (1 - p_1)

In [ ]:
def Q(data_left, data_right):
    N_l = left.shape[0]
    N_r = right.shape[0]
    N_m = N_l + N_r
    return N_l / N_m * H(data_left) + N_r / N_m * H(data_right)

In [ ]:
def choose_best(data):
    for i in range(1, data.shape[0]):

## Строим решающее дерево для небольшой выборки регрессии

**Критерии информативности в задачах регрессии**

*Mean squared error:*
$$H = \frac{1}{n}\sum_i (y_i - \hat{y})^2$$
где $\hat{y}$ – средне по $i$-ым объектам
$n$ – количество объектов.


---

*Mean absolute error:*
$$H =  \frac{1}{n}\sum_i |y - \hat{y}|$$
где $\hat{y}$ – медиана по $i$-ым объектам

* Построить решающее дерево глубины 2 для задачи регрессии аналитически (посчитав критерий информативности для всех возможных сплитов на листочке или с помощью функции на python);
* Сравнить полученный результат с реализацией в sklearn с помощью `plot_tree`.

In [ ]:
X = np.array([
    [1],
    [2],
    [3],
    [4],
    [5]
])

y = [3, 2.5, 2.0, 2.2, 2.7]

plot_reg_dataset(X, y,  figsize=FIG_SIZE)